In [30]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
from math import sqrt

from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor

In [31]:
data = pd.read_csv('Resources/Clean_Data/Final_Data.csv')
data.head()

,Zip_Code,Date,Date_Code,Year,Month,Sale_Price,Interest_Rate,Property_Tax,Rent_Price,Household_Income,...,FTE_Employed,Unemployed,Expense_Index,Average_Commute,Crime_Index,Loan_Amount,Loan_Term,Loan_R,Loan_Payment,Home_Affordability
0,32003,2019-01-01,201901,2019,1,244950.0,4.4640,3778,1113,94154.0,...,10671,1069,104,35,28,195960.0,360.0,1.003720,988.713278,0.126012
1,32003,2019-02-01,201902,2019,2,270000.0,4.3700,3778,1113,94154.0,...,10671,1069,104,35,28,216000.0,360.0,1.003642,1077.819242,0.137369
2,32003,2019-03-01,201903,2019,3,275000.0,4.2650,3778,1113,94154.0,...,10671,1069,104,35,28,220000.0,360.0,1.003554,1084.200547,0.138182
3,32003,2019-04-01,201904,2019,4,264500.0,4.1425,3778,1113,94154.0,...,10671,1069,104,35,28,211600.0,360.0,1.003452,1027.671397,0.130978
4,32003,2019-05-01,201905,2019,5,281000.0,4.0720,3778,1113,94154.0,...,10671,1069,104,35,28,224800.0,360.0,1.003393,1082.581718,0.137976


In [32]:
ml_data = data[['Zip_Code', 'Date_Code', 'Mobility_Rate', 'Expense_Index', 'Crime_Index' , 'Total_Vacant', 'Total_Dwellings', 'Total_Sales', 'FHA_Count', 'Home_Affordability', 'Rent_Affordability', 'Sale_Price' ]]
ml_data.head()

,Zip_Code,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
0,32003,201901,0.182327,104,28,704,11724,32,7,0.126012,0.141853,244950.0
1,32003,201902,0.182327,104,28,704,11724,41,2,0.137369,0.141853,270000.0
2,32003,201903,0.182327,104,28,704,11724,21,7,0.138182,0.141853,275000.0
3,32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0
4,32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0


In [33]:
rf_data = ml_data.sort_values(by = ['Zip_Code','Date_Code'], ascending = [True, True])
rf_data['Zip_Code'] = rf_data['Zip_Code'].astype(str)
rf_data = rf_data.set_index('Zip_Code')


rf_data

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price
Zip_Code,,,,,,,,,,,
32003,201901,0.182327,104,28,704,11724,32,7,0.126012,0.141853,244950.0
32003,201902,0.182327,104,28,704,11724,41,2,0.137369,0.141853,270000.0
32003,201903,0.182327,104,28,704,11724,21,7,0.138182,0.141853,275000.0
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0
...,...,...,...,...,...,...,...,...,...,...,...
34997,202011,0.143471,91,113,2976,21551,132,11,0.173497,0.247189,259750.0
34997,202012,0.143471,91,113,2976,21551,142,16,0.196301,0.247189,297000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0


In [34]:
rf_data['Last_Month_Price'] = rf_data.groupby(['Zip_Code'])['Sale_Price'].shift()
rf_data['Last_2Month_Price'] = rf_data.groupby(['Zip_Code'])['Sale_Price'].shift(2)
rf_data['Last_3Month_Price'] = rf_data.groupby(['Zip_Code'])['Sale_Price'].shift(3)
# ml_data_3mo['Future_Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-1)
# ml_data_3mo['Future_2Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-2)
# ml_data_3mo['Future_3Month_Price'] = ml_data_3mo.groupby(['Zip_Code'])['Sale_Price'].shift(-3)


price_difference_1 = []
price_difference_2 = []
price_difference_3 = []


for column in rf_data[['Sale_Price', 'Last_Month_Price', 'Last_2Month_Price',
                    'Last_3Month_Price']]:
   # Select column contents by column name using [] operator
    current_sales = rf_data['Sale_Price'].values
    last_sales = rf_data['Last_Month_Price'].values
    last_2mo_sale = rf_data['Last_2Month_Price'].values
    last_3mo_sale = rf_data['Last_3Month_Price'].values
    price_difference_1.append(current_sales-last_sales)
    price_difference_2.append(last_sales-last_2mo_sale)
    price_difference_3.append(last_2mo_sale-last_3mo_sale)
    

rf_data['Last_Month_Diff'] = price_difference_1[0]
rf_data['Last_2Month_Diff'] = price_difference_2[0]
rf_data['Last_3Month_Diff'] = price_difference_3[0]

rf_data

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201901,0.182327,104,28,704,11724,32,7,0.126012,0.141853,244950.0,NaN,NaN,NaN,NaN,NaN,NaN
32003,201902,0.182327,104,28,704,11724,41,2,0.137369,0.141853,270000.0,244950.0,NaN,NaN,25050.0,NaN,NaN
32003,201903,0.182327,104,28,704,11724,21,7,0.138182,0.141853,275000.0,270000.0,244950.0,NaN,5000.0,25050.0,NaN
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34997,202011,0.143471,91,113,2976,21551,132,11,0.173497,0.247189,259750.0,285000.0,256000.0,251000.0,-25250.0,29000.0,5000.0
34997,202012,0.143471,91,113,2976,21551,142,16,0.196301,0.247189,297000.0,259750.0,285000.0,256000.0,37250.0,-25250.0,29000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0


In [35]:
rf_data.replace([np.inf, -np.inf], np.nan, inplace=True)
rf_data.dropna(inplace=True)
rf_data.head()

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0


In [36]:
rf_data['Date_Code'] = rf_data['Date_Code'].astype(str)
rf_data

,Date_Code,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,,
32003,201904,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,201905,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,201906,0.182327,104,28,704,11724,43,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,201907,0.182327,104,28,704,11724,90,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,201908,0.182327,104,28,704,11724,64,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34997,202011,0.143471,91,113,2976,21551,132,11,0.173497,0.247189,259750.0,285000.0,256000.0,251000.0,-25250.0,29000.0,5000.0
34997,202012,0.143471,91,113,2976,21551,142,16,0.196301,0.247189,297000.0,259750.0,285000.0,256000.0,37250.0,-25250.0,29000.0
34997,202101,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0


In [37]:
test = rf_data[rf_data['Date_Code'].str.contains('2021')].drop(columns=['Date_Code'])
test

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,0.182327,104,28,704,11724,39,6,0.124589,0.141853,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0
32003,0.182327,104,28,704,11724,30,8,0.140027,0.141853,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0
32003,0.182327,104,28,704,11724,70,4,0.147681,0.141853,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0
32008,0.067037,82,118,964,3247,6,0,0.153378,0.285180,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0
32008,0.067037,82,118,964,3247,3,1,0.062420,0.285180,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34996,0.141276,78,140,2780,9249,57,1,0.225364,0.209918,395000.0,345000.0,390000.0,400000.0,50000.0,-45000.0,-10000.0
34996,0.141276,78,140,2780,9249,14,0,0.275304,0.209918,465950.0,395000.0,345000.0,390000.0,70950.0,50000.0,-45000.0
34997,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0


In [38]:
y_test = test['Sale_Price'].values

X_test = test.drop(columns='Sale_Price')

feature_test_names = X_test.columns

X_test.head()

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,
32003,0.182327,104,28,704,11724,39,6,0.124589,0.141853,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0
32003,0.182327,104,28,704,11724,30,8,0.140027,0.141853,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0
32003,0.182327,104,28,704,11724,70,4,0.147681,0.141853,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0
32008,0.067037,82,118,964,3247,6,0,0.153378,0.285180,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0
32008,0.067037,82,118,964,3247,3,1,0.062420,0.285180,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0


In [39]:
training = rf_data[~rf_data['Date_Code'].str.contains('2021')].drop(columns=['Date_Code'])
training

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,,
32003,0.182327,104,28,704,11724,42,3,0.130978,0.141853,264500.0,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,0.182327,104,28,704,11724,66,8,0.137976,0.141853,281000.0,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,0.182327,104,28,704,11724,43,3,0.130690,0.141853,275000.0,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,0.182327,104,28,704,11724,90,7,0.133641,0.141853,282500.0,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,0.182327,104,28,704,11724,64,5,0.125212,0.141853,269577.0,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34997,0.143471,91,113,2976,21551,123,18,0.171370,0.247189,251000.0,255000.0,259900.0,240000.0,-4000.0,-4900.0,19900.0
34997,0.143471,91,113,2976,21551,115,18,0.173776,0.247189,256000.0,251000.0,255000.0,259900.0,5000.0,-4000.0,-4900.0
34997,0.143471,91,113,2976,21551,139,17,0.192070,0.247189,285000.0,256000.0,251000.0,255000.0,29000.0,5000.0,-4000.0


In [40]:
y_train= training['Sale_Price'].values

X_train = training.drop(columns='Sale_Price')

feature_train_names = X_train.columns

X_train.head()

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff
Zip_Code,,,,,,,,,,,,,,,
32003,0.182327,104,28,704,11724,42,3,0.130978,0.141853,275000.0,270000.0,244950.0,-10500.0,5000.0,25050.0
32003,0.182327,104,28,704,11724,66,8,0.137976,0.141853,264500.0,275000.0,270000.0,16500.0,-10500.0,5000.0
32003,0.182327,104,28,704,11724,43,3,0.130690,0.141853,281000.0,264500.0,275000.0,-6000.0,16500.0,-10500.0
32003,0.182327,104,28,704,11724,90,7,0.133641,0.141853,275000.0,281000.0,264500.0,7500.0,-6000.0,16500.0
32003,0.182327,104,28,704,11724,64,5,0.125212,0.141853,282500.0,275000.0,281000.0,-12923.0,7500.0,-6000.0


In [19]:
#Random Forest Regressor
rf_regr = RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)
rf_regr.fit(X_train, y_train)
# np.log1p(ytr)

RandomForestRegressor(n_estimators=1000, n_jobs=-1, random_state=42)

In [13]:
print(f"Training Data Score: {rf_regr.score(X_train, y_train)}")
print(f"Testing Data Score: {rf_regr.score(X_test, y_test)}")

Training Data Score: 0.9981845826685722
Testing Data Score: 0.978534981036313


In [47]:
rf_pred = rf_regr.predict(X_test)

print('Mean Error = %.5f' % np.sqrt(mean_squared_log_error(y_test, rf_pred)))

Mean Error = 0.07004


In [61]:
import math
MSE = mean_squared_error(y_test, rf_pred))
 
RMSE = math.sqrt(MSE)

print(MSE)
print("Root Mean Square Error:\n")
print(RMSE)

613775079.1469097
Root Mean Square Error:

24774.484437560142


In [58]:
mean_error = []
for row in test.iterrows():
    # train = melt2[melt2['Week'] < week]
    # val = melt2[melt2['Week'] == week]
    prev = test['Last_3Month_Price'].values
    curr = test['Sale_Price'].values

    error = np.sqrt(mean_squared_log_error(curr, prev))
    print('Week  - Error %.5f' % (error))
    mean_error.append(error)
# print('Mean Error = %.5f' % np.mean(mean_error))

Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Error 0.32047
Week  - Er

In [ ]:
rmse = []
for row in test():



In [118]:
output = pd.DataFrame({'Predicted':rf_pred, 'Actual': y_test})
output

,Predicted,Actual
0,300813.433,299900.0
1,333409.450,333800.0
2,339040.001,339950.0
3,133453.210,132000.0
4,65845.209,53200.0
...,...,...
2537,395403.237,395000.0
2538,464766.475,465950.0
2539,305500.998,305000.0
2540,300436.306,300000.0


In [63]:
test['Predicted_Sale_Price'] = rf_pred
test

,Mobility_Rate,Expense_Index,Crime_Index,Total_Vacant,Total_Dwellings,Total_Sales,FHA_Count,Home_Affordability,Rent_Affordability,Sale_Price,Last_Month_Price,Last_2Month_Price,Last_3Month_Price,Last_Month_Diff,Last_2Month_Diff,Last_3Month_Diff,Predicted_Sale_Price
Zip_Code,,,,,,,,,,,,,,,,,
32003,0.182327,104,28,704,11724,39,6,0.124589,0.141853,299900.0,294000.0,336000.0,330000.0,5900.0,-42000.0,6000.0,300813.433
32003,0.182327,104,28,704,11724,30,8,0.140027,0.141853,333800.0,299900.0,294000.0,336000.0,33900.0,5900.0,-42000.0,333409.450
32003,0.182327,104,28,704,11724,70,4,0.147681,0.141853,339950.0,333800.0,299900.0,294000.0,6150.0,33900.0,5900.0,339040.001
32008,0.067037,82,118,964,3247,6,0,0.153378,0.285180,132000.0,169500.0,72500.0,107000.0,-37500.0,97000.0,-34500.0,133453.210
32008,0.067037,82,118,964,3247,3,1,0.062420,0.285180,53200.0,132000.0,169500.0,72500.0,-78800.0,-37500.0,97000.0,65845.209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34996,0.141276,78,140,2780,9249,57,1,0.225364,0.209918,395000.0,345000.0,390000.0,400000.0,50000.0,-45000.0,-10000.0,395403.237
34996,0.141276,78,140,2780,9249,14,0,0.275304,0.209918,465950.0,395000.0,345000.0,390000.0,70950.0,50000.0,-45000.0,464766.475
34997,0.143471,91,113,2976,21551,81,10,0.202930,0.247189,305000.0,297000.0,259750.0,285000.0,8000.0,37250.0,-25250.0,305500.998


In [41]:
mdl = LGBMRegressor(n_estimators=1000, learning_rate=0.01)
mdl.fit(X_train, y_train)

LGBMRegressor(learning_rate=0.01, n_estimators=1000)

In [43]:
p = mdl.predict(X_test)
p

array([303563.00814857, 332431.60482848, 337677.59991545, ...,
       303481.02690579, 298364.68309304, 546370.9883532 ])

In [44]:
print(f"Training Data Score: {mdl.score(X_train, y_train)}")
print(f"Testing Data Score: {mdl.score(X_test, y_test)}")

Training Data Score: 0.9948551023182268
Testing Data Score: 0.9744681867783782


In [45]:
lgbm_pred = mdl.predict(X_test)

print('Mean Error = %.5f' % np.sqrt(mean_squared_log_error(y_test, rf_pred)))

Mean Error = 0.07004


In [109]:
values = X_test.values
row = values[-1:]
# make a one-step prediction
yhat = rf_regr.predict(row)
print('Input: %s, Predicted: %.3f' % (row, yhat[0]))

Input: [[ 1.43471312e-01  9.10000000e+01  1.13000000e+02  2.97600000e+03
   2.15510000e+04  7.00000000e+00  1.20000000e+01  4.16753590e-01
   2.47189100e-01  3.00000000e+05  3.05000000e+05  2.97000000e+05
   2.99000000e+05 -5.00000000e+03  8.00000000e+03]], Predicted: 559847.963


In [107]:
# output2 = pd.DataFrame({'Predicted':yhat[0]})
# output2
yhat[0]

305500.998

In [100]:
zip_codes = rf_data.index.unique()

Index(['32003', '32008', '32009', '32011', '32024', '32025', '32033', '32034',
       '32038', '32040',
       ...
       '34981', '34982', '34983', '34984', '34986', '34987', '34990', '34994',
       '34996', '34997'],
      dtype='object', name='Zip_Code', length=898)